<a href="https://colab.research.google.com/github/Kaleab1999/Ham-Spam-classifier/blob/main/uni_gram_with_TF_IDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## spam checker

importing pandas to load our dataset

In [1]:
import pandas as pd

1. loading the dataset
2. separate the dataset by 't' or tab
3. label the separatted data 

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
df=pd.read_csv('/content/drive/MyDrive/dataset/ssms_spam.txt',sep='\t',
                           names=["label", "message"])
df.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


## Data cleaning

In [4]:
import re #for lowerring text, and maintaining words
import nltk #for NLP (to perform tokenization,stop_words,stemming,lemittization,bag_of_words,TFIDF)

In [5]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
from nltk.corpus import stopwords # to apply stopwords
from nltk.stem.porter import PorterStemmer #for stemming
from nltk.stem import WordNetLemmatizer #to perform Lemmatizzation

creating object for stemming and lemmatization

In [7]:
ps = PorterStemmer()
corpus = []

In [8]:
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [9]:
for i in range(0, len(df)):
    review2 = re.sub('[^a-zA-Z]', ' ', df['message'][i])#to remove all except letters
    review2 = review2.lower() #lowerring the letter
    review2 = review2.split() #tokenize
    
    review2 = [ps.stem(word) for word in review2 if not word in stopwords.words('english')]
    review2 = ' '.join(review2)
    corpus.append(review2)

In [10]:
import textblob
from textblob import TextBlob

In [11]:
df['polarity']=df['message'].apply(lambda tweet:TextBlob(tweet).sentiment.polarity)
df['subjectivity']=df['message'].apply(lambda tweet:TextBlob(tweet).sentiment.subjectivity)

df.drop(['message'], axis=1, inplace=True)
df.head()


,label,polarity,subjectivity
0,ham,0.15,0.7625
1,ham,0.50,0.5000
2,spam,0.30,0.5500
3,ham,0.10,0.3000
4,ham,0.00,0.0000


### Creating the TF-IDF model

In [23]:

from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(max_features=5000, ngram_range=(1,1))
X = tf.fit_transform(corpus).toarray()

y=pd.get_dummies(df['label'])
y=y.iloc[:,1].values

In [24]:
y= pd.DataFrame(y)
y.head()

,0
0,0
1,0
2,1
3,0
4,0


In [25]:
x=pd.DataFrame(X)
k=df[['polarity','subjectivity']]
p=pd.DataFrame(k)
X=pd.concat([x,p],axis=1)

X.head()

,0,1,2,3,4,5,6,7,8,9,...,4992,4993,4994,4995,4996,4997,4998,4999,polarity,subjectivity
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.15,0.7625
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.50,0.5000
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.30,0.5500
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.10,0.3000
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0000


splitting the dataset into train and test

In [26]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)

In [27]:
# Training model 

from sklearn.ensemble import RandomForestClassifier

randomclassifier=RandomForestClassifier(n_estimators=200,criterion='entropy')
randomclassifier.fit(X_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestClassifier(criterion='entropy', n_estimators=200)

In [28]:
## Predict for the Test Dataset

predictions = randomclassifier.predict(X_test)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


In [29]:
y_test.columns

RangeIndex(start=0, stop=1, step=1)

In [30]:
## Import library to check accuracy
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

matrix=confusion_matrix(y_test[0],predictions)
print(matrix)
score=accuracy_score(y_test[0],predictions)
print(score)
report=classification_report(y_test[0],predictions)
print(report)

[[966   0]
 [ 23 126]]
0.979372197309417
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       966
           1       1.00      0.85      0.92       149

    accuracy                           0.98      1115
   macro avg       0.99      0.92      0.95      1115
weighted avg       0.98      0.98      0.98      1115



In [31]:
from sklearn.linear_model import  LogisticRegression

logreg=LogisticRegression(C=1e5, max_iter=10000 )
logreg.fit(X_train, y_train)

pred = logreg.predict(X_test)

from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

matrix=confusion_matrix(y_test[0],pred)
print(matrix)
score=accuracy_score(y_test[0],pred)
print(score)
report=classification_report(y_test[0],pred)
print(report)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[[963   3]
 [ 17 132]]
0.9820627802690582
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       966
           1       0.98      0.89      0.93       149

    accuracy                           0.98      1115
   macro avg       0.98      0.94      0.96      1115
weighted avg       0.98      0.98      0.98      1115



/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


In [32]:
randomclassifier=RandomForestClassifier(n_estimators=200)
randomclassifier.fit(X_train,y_train)

pred= randomclassifier.predict(X_test)

from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

matrix=confusion_matrix(y_test[0],pred)
print(matrix)
score=accuracy_score(y_test[0],pred)
print(score)
report=classification_report(y_test[0],pred)
print(report)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


[[966   0]
 [ 23 126]]
0.979372197309417
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       966
           1       1.00      0.85      0.92       149

    accuracy                           0.98      1115
   macro avg       0.99      0.92      0.95      1115
weighted avg       0.98      0.98      0.98      1115



/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


In [33]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train, y_train)

pred= gnb.predict(X_test)

from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

matrix=confusion_matrix(y_test[0],pred)
print(matrix)
score=accuracy_score(y_test[0],pred)
print(score)
report=classification_report(y_test[0],pred)
print(report)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


[[835 131]
 [ 25 124]]
0.8600896860986547
              precision    recall  f1-score   support

           0       0.97      0.86      0.91       966
           1       0.49      0.83      0.61       149

    accuracy                           0.86      1115
   macro avg       0.73      0.85      0.76      1115
weighted avg       0.91      0.86      0.87      1115



/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
